# Wagner Database AI Extraction

#### Code works best with a python version 3.10 < version < 3.12 (I am using 3.10.13)

In [1]:
from enum import Enum
from typing import Optional, List
import os
from openai import OpenAI
import json
import asyncio
import base64
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/williamharrigan/Desktop/UH/Year_3/semester_2/wagner')
import creds

## pydantic
from pydantic import BaseModel, Field
from openai.lib._pydantic import to_strict_json_schema
from pydantic_ai import Agent
from openai.types.chat.chat_completion_content_part_param import (
    ChatCompletionContentPartTextParam,
    ChatCompletionContentPartImageParam
)

from openai.types.chat.chat_completion_content_part_image_param import (
    ImageURL
)

photo_dir = "../training_set/"

In [2]:
class Description(str, Enum):
    DICOTS = "Dicots"
    MONOCOTS = "Monocots"
    CONIFERS = "Conifers"
    FERNS = "Ferns and fern allies"

class StemHairType(str, Enum):
    DENDRITIC = "DENDRITIC"
    GLABROUS = "GLABROUS"
    HIRSUTE = "HIRSUTE"
    HISPID = "HISPID"
    LEPIDOTE = "LEPIDOTE"
    PILOSE = "PILOSE"
    PUBERULENT = "PUBERULENT"
    STRIGOSE = "STRIGOSE"
    STELLATE = "STELLATE"
    TOMENTOSE = "TOMENTOSE"
    VILLOUS = "VILLOUS"
    GLAUCOUS = "GLAUCOUS"
    
class FruitType(str, Enum):
    ACHENE = "ACHENE"
    AGGREGATE = "AGGREGATE"
    ARTICLE = "ARTICLE"
    BERRY = "BERRY"
    CAPSULE = "CAPSULE"
    CARYOPSIS = "CARYOPSIS"
    DRUPE = "DRUPE"
    FOLLICLE = "FOLLICLE"
    LEGUME = "LEGUME"
    MERICARP = "MERICARP"
    MULTIPLE = "MULTIPLE"
    NUT = "NUT"
    PEPO = "PEPO"
    POME = "POME"
    SCHIZOCARP = "SCHIZOCARP"
    SILICLE = "SILICLE"
    SILIQUE = "SILIQUE"
    SYCONIUM = "SYCONIUM"
    
class LeafShapeType(str, Enum):
    ACEROSE = "ACEROSE"
    AWL_SHAPED = "AWL_SHAPED"
    GLADIATE = "GLADIATE"
    HASTATE = "HASTATE"
    CORDATE = "CORDATE"
    DELTOID = "DELTOID"
    LANCEOLATE = "LANCEOLATE"
    LINEAR = "LINEAR"
    ELLIPTIC = "ELLIPTIC"
    ENSIFORM = "ENSIFORM"
    LYRATE = "LYRATE"
    OBCORDATE = "OBCORDATE"
    FALCATE = "FALCATE"
    FLABELLATE = "FLABELLATE"
    OBDELTOID = "OBDELTOID"
    OBELLIPTIC = "OBELLIPTIC"
    OBLANCEOLATE = "OBLANCEOLATE"
    OBLONG = "OBLONG"
    PERFOLIATE = "PERFOLIATE"
    QUADRATE = "QUADRATE"
    OBOVATE = "OBOVATE"
    ORBICULAR = "ORBICULAR"
    RENIFORM = "RENIFORM"
    RHOMBIC = "RHOMBIC"
    OVAL = "OVAL"
    OVATE = "OVATE"
    ROTUND = "ROTUND"
    SAGITTATE = "SAGITTATE"
    PANDURATE = "PANDURATE"
    PELTATE = "PELTATE"
    SPATULATE = "SPATULATE"
    SUBULATE = "SUBULATE"

class StemHairs(str, Enum):
    DENDRITIC = "DENDRITIC"
    GLABROUS = "GLABROUS"

class PhyllotaxyType(str, Enum):
    ALTERNATE = "ALTERNATE"
    OPPOSITE = "OPPOSITE"
    WHORLED = "WHORLED"
    DECUSSATE = "DECUSSATE"
    DISTICHOUS = "DISTICHOUS"
    EQUITANT = "EQUITANT"
    TERNATE = "TERNATE"
    CAULINE = "CAULINE"

class InflorescenceType(str, Enum):
    CATKIN = "CATKIN"
    CYME = "CYME"
    HEAD = "HEAD"
    PANICLE = "PANICLE"
    RACEME = "RACEME"
    SPATHE_SPADIX = "SPATHE_SPADIX"
    THYRSE = "THYRSE"
    UMBEL = "UMBEL"
    VERTISCILLATE = "VERTISCILLATE"
    SOLITARY = "SOLITARY"
    SPIKE = "SPIKE"
    LANCEOLATE = "LANCEOLATE"
    GLOBOSE = "GLOBOSE"
    INVOLUCRE = "INVOLUCRE"
    CORYMBOSE = "CORYMBOSE"
    STROBILOID = "STROBILOID"

class LeafHairType(str, Enum):
    DENDRITIC = "DENDRITIC"
    GLABROUS = "GLABROUS"
    HIRSUTE = "HIRSUTE"
    HISPID = "HISPID"
    LEPIDOTE = "LEPIDOTE"
    PILOSE = "PILOSE"
    PUBERULENT = "PUBERULENT"
    STRIGOSE = "STRIGOSE"
    STELLATE = "STELLATE"
    TOMENTOSE = "TOMENTOSE"
    VILLOUS = "VILLOUS"
    GLAUCOUS = "GLAUCOUS"

class LeafType(str, Enum):
    SIMPLE = "SIMPLE"
    COMPOUND = "COMPOUND"

class BreedingType(str, Enum):
    MONOECIOUS = "MONOECIOUS"  # Male and female flowers on the same plant
    ANDROMONOECIOUS = "ANDROMONOECIOUS"
    CHASMOGAMOUS = "CHASMOGAMOUS"
    DIOECIOUS = "DIOECIOUS"  # Separate male and female plants
    GYNODIOECIOUS = "GYNODIOECIOUS"
    POLYGAMO_MONOECIOUS = "POLYGAMO-MONOECIOUS"
    POLYGAMOUS = "POLYGAMOUS"
    POLYGAMO_DIOECIOUS = "POLYGAMO-DIOECIOUS"
    GYNOMONOECIOUS = "GYNOMONOECIOUS"
    STERILE = "STERILE"

class LeafMarginType(str, Enum):
    TEETH = "TEETH"
    LOBED = "LOBED"
    ENTIRE = "ENTIRE"


class LifeFormType(str, Enum):
    ANNUAL_HERB = "ANNUAL_HERB"
    PERENNIAL_HERB = "PERENNIAL_HERB"
    EPIPHYTE = "EPIPHYTE"
    VINE = "VINE"
    SHRUB = "SHRUB"
    TREE = "TREE"

class CorollaType(str, Enum):
    ADNATE = "ADNATE"
    BILABIATE = "BILABIATE"
    CAMPANULATE = "CAMPANULATE"
    CORYMBOSE = "CORYMBOSE"
    CONVOLUTE = "CONVOLUTE"
    CORONA = "CORONA"
    CUNEATE = "CUNEATE"
    CYLINDRICAL = "CYLINDRICAL"
    DISK = "DISK"
    DELTATE = "DELTATE"
    ELLIPTIC = "ELLIPTIC"
    FUNNELFORM = "FUNNELFORM"
    FLABELLATE = "FLABELLATE"
    FILIFORM = "FILIFORM"
    HOOD = "HOOD"
    IRREGULAR = "IRREGULAR"
    KEEL = "KEEL"
    LABELLUM = "LABELLUM"
    LANCEOLATE = "LANCEOLATE"
    LINEAR = "LINEAR"
    LIPPED = "LIPPED"
    LOBBED = "LOBBED"
    OVATE = "OVATE"
    OBLONG = "OBLONG"
    OBCORDATE = "OBCORDATE"
    OBOVATE = "OBOVATE"
    OBLANCEOLATE = "OBLANCEOLATE"
    ORBICULAR = "ORBICULAR"
    PALATE = "PALATE"
    PSEUDORACEMES = "PSEUDORACEMES"
    ROTATE = "ROTATE"
    RAY = "RAY"
    REFLEXED = "REFLEXED"
    RHOMBIC = "RHOMBIC"
    RENIFORM = "RENIFORM"
    SALVERFORM = "SALVERFORM"
    SUBORBICULAR = "SUBORBICULAR"
    SUBRHOMBIC = "SUBRHOMBIC"
    SPUR = "SPUR"
    SPATULATE = "SPATULATE"
    SPICATE = "SPICATE"
    SUBROTATE = "SUBROTATE"
    STANDARD = "STANDARD"
    TUBULAR = "TUBULAR"
    TRIANGULAR = "TRIANGULAR"
    URCEOLATE = "URCEOLATE"
    UNILABIATE = "UNILABIATE"
    VALVATE = "VALVATE"
    VERTICIL = "VERTICIL"
    ZYGOMORPHIC = "ZYGOMORPHIC"
    CUP = "CUP"
    UNGUICULATE = "UNGUICULATE"
    CLAW = "CLAW"
    FASICLE = "FASICLE"
    STELLATE = "STELLATE"
    SUBPANICULATE = "SUBPANICULATE"
    PENTAGONAL = "PENTAGONAL"

class OriginType(str, Enum):
    NATURALIZED = "NATURALIZED"
    INDIGENOUS = "INDIGENOUS"
    ENDEMIC = "ENDEMIC"
    POLYNESIAN_INTRODUCTION = "POLYNESIAN INTRODUCTION"

class Location(str, Enum):
    HAWAII = "HAWAII"
    MAUI = "MAUI"
    KAHOOLAWE = "KAHOOLAWE"
    MOLOKAI = "MOLOKAI"
    LANAI = "LANAI"
    OAHU = "OAHU"
    KAUAI = "KAUAI"
    NIIHAU = "NIIHAU"
    ALL_ISLANDS = "ALL ISLANDS"

class FederalStatusType(str, Enum):
    SPECIES_OF_CONCERN = "SPECIES_OF_CONCERN"
    ENDANGERED = "ENDANGERED"
    THREATENED = "THREATENED"
    WITHDRAWN = "WITHDRAWN" 
    
class StatusType(str, Enum):
    NATURALIZED = "NATURALIZED"
    ENDEMIC = "ENDEMIC"
    RARE = "RARE"
    SECURE = "SECURE"
    VULNERABLE = "VULNERABLE"

class Measurements(BaseModel):
    min: Optional[float] = None
    max: Optional[float] = None
    extreme_min: Optional[float] = None
    extreme_max: Optional[float] = None

class HawaiianPlant(BaseModel):
    # Basic Information
    family: str = Field(..., description="Plant family name (should only be 1 object)")
    genus: str = Field(..., description="Plant genus name (should only be 1 object)")
    species: str = Field(..., description="Plant species name (should only be 1 object)")
    common_name: Optional[str] = Field(None, description="Common name of the plant")
    wagner_pg_number: Optional[str] = Field(None, description="Wagner book reference number")
    description: Optional[Description] = Field(None, description="Take knowledge from outside the passage to infer whether the plant is DICOTS, MONOCOTS, CONIFERS or FERNS")
    infraspecific_epithet: str = Field(..., description="The third word in the scientific name of an infraspecific taxon, following the name of the species. This applies only to formal names of plants and fungi, and not to the formal names of bacteria or animals. In the name Cannabis sativa subsp. indica, the word indica is the infraspecific epithet.")

    hawaiian_name: Optional[List[str]] = Field(None, description="List of Hawaiian names")
    
    stem_hair_type: Optional[StemHairType] = Field(None, description="Type of hair on stem")
    
    phyllotaxy_type: Optional[PhyllotaxyType] = Field(None, description="The arrangement of leaves around the stem.")

    leaf_hair_description: Optional[str] = Field(None, description="Description of leaf hair.")
    leaf_hair_upper_description: Optional[str] = Field(None, description="Description of Upper leaf hairs.")
    leaf_hair_lower_description: Optional[str] = Field(None, description="Description of Lower leaf hairs.")
    
    breeding_type: Optional[BreedingType] = Field(None, description="Plant reproductive class.")
    
    inflorescence_type: Optional[InflorescenceType] = Field(None, description="In a flowering plant, a cluster of flowers on a branch or a system of branches")
    
    ray_color: Optional[str] = Field(None, description="Color of ray")
    floret_color: Optional[str] = Field(None, description="Color of florets")
    spathe_color: Optional[str] = Field(None, description="Color of spathe")
    perianth_outer_color: Optional[str] = Field(None, description="Color of perianth outer flower")
    perianth_inner_color: Optional[str] = Field(None, description="Color of perianth inner flower")
    perianth_color: Optional[str] = Field(None, description="Color of perianth")
    labellum_color: Optional[str] = Field(None, description="Color of labellum")

    corolla_type: Optional[CorollaType] = Field(None, description="Type of corolla")
    corolla_color: Optional[str] = Field(None, description="Color of corolla")
    staminate_corolla_type: Optional[CorollaType] = Field(None, description="Type of staminate corolla")
    pistillate_corolla_type: Optional[CorollaType] = Field(None, description="Type of pistillate corolla")
    
    fruit_type: Optional[FruitType] = Field(None, description="Type of fruit")
    fruit_length: Optional[Measurements] = Field(None, description="Fruit length in millimeters")
    fruit_width: Optional[Measurements] = Field(None, description="Fruit width in millimeters")
    fruit_diameter: Optional[Measurements] = Field(None, description="Fruit diameter in millimeters")
    
    ploidy: Optional[str] = Field(None, description="Ploidy level expressed as a function of n (e.g., 1n, 2n or 3n, etc..)")
    chromosome_number: Optional[int] = Field(None, description="The integer Number of chromosomes")
    average_chromosome_number: Optional[float] = Field(None, description="Average chromosome number")
    
    origin: Optional[OriginType] = Field(None, description="Origin type of the plant")
    fed_status: Optional[FederalStatusType] = Field(None, description="Federal conservation status")
    status: Optional[StatusType] = Field(None, description="General status")

    life_form_type: Optional[LifeFormType] = Field(None, description="Growth habit or life form")
    leaf_type: Optional[LeafType] = Field(None, description="Simple or compound leaf type")
    leaf_shape_type: Optional[LeafShapeType] = Field(None, description="Shape of leaves")
    leaf_margin_type: Optional[LeafMarginType] = Field(None, description="Type of leaf margin")
    
    juvenile_leaf_type: Optional[LeafType] = Field(None, description="Simple or compound juvenile leaf type")
    juvenile_leaf_shape_type: Optional[LeafShapeType] = Field(None, description="Shape of juvenile leaves")
    juvenile_leaf_margin_type: Optional[LeafMarginType] = Field(None, description="Type of leaf margin on juvenile plants")
    
    leaflets_leaf_type: Optional[LeafType] = Field(None, description="Simple or compound juvenile leaf type")
    leaflets_leaf_shape_type: Optional[LeafShapeType] = Field(None, description="Shape of juvenile leaves")
    leaflets_leaf_margin_type: Optional[LeafMarginType] = Field(None, description="Type of leaf margin on juvenile plants")
    
    leaf_hair_upper_type: Optional[LeafHairType] = Field(None, description="Type of upper leaf hairs")
    leaf_hair_lower_type: Optional[LeafHairType] = Field(None, description="Type of lower leaf hairs")
    leaf_hair_type: Optional[LeafHairType] = Field(None, description="Type of leaf hairs")
    juvenile_leaf_hair_type: Optional[LeafHairType] = Field(None, description="Type of juvenile leaf hairs")
    
    island_type: Optional[List[Location]] = Field(None, description="Islands where the plant is found")
    
    stem_height: Optional[Measurements] = Field(None, description="Stem or general plant height measurements in meters")
    leaf_length: Optional[Measurements] = Field(None, description="Length of leaves in millimeters")
    leaf_width: Optional[Measurements] = Field(None, description="Width of leaves in millimeters")
    juvenile_leaf_length: Optional[Measurements] = Field(None, description="Juvenile length of leaves in millimeters")
    juvenile_leaf_width: Optional[Measurements] = Field(None, description="Juvenile width of leaves in millimeters")
    leaflets_leaf_length: Optional[Measurements] = Field(None, description="Leaflets length of leaves in millimeters")
    leaflets_leaf_width: Optional[Measurements] = Field(None, description="Leaflets width of leaves in millimeters")    
    
    petioles: Optional[Measurements] = Field(None, description="Length of petiole stalk in centimeters") 
    staminate_inflorescence_length: Optional[Measurements] = Field(None, description="The measured length in millimeters of the male (pollen-producing) flower cluster. This specifically refers to catkins or other inflorescences containing only staminate (male) flowers.")    
    pistillate_inflorescence_length: Optional[Measurements] = Field(None, description="The measured length of the female (seed-producing) flower cluster. This specifically refers to inflorescences containing only pistillate (female) flowers.")    
    staminate_inflorescence_width: Optional[Measurements] = Field(None, description="The measured width in millimeters of the male (pollen-producing) flower cluster. This specifically refers to catkins or other inflorescences containing only staminate (male) flowers.")    
    pistillate_inflorescence_width: Optional[Measurements] = Field(None, description="The measured width of the female (seed-producing) flower cluster. This specifically refers to inflorescences containing only pistillate (female) flowers.")        
    
    inflorescence_flower_length: Optional[Measurements] = Field(None, description="The length of an inflorescence flower in millimeters.")
    inflorescence_flower_width: Optional[Measurements] = Field(None, description="The width of an inflorescence flower in millimeters.")
    
    flower_length: Optional[Measurements] = Field(None, description="Flower length in centimeters")
    flower_width: Optional[Measurements] = Field(None, description="Flower width in centimeters")
    
    rachis_length: Optional[Measurements] = Field(None, description="Rachis length in millimeters")
    rachis_diameter: Optional[Measurements] = Field(None, description="Rachis diameter in millimeters") 
    
    head_length: Optional[Measurements] = Field(None, description="The measured length of the capitulum (flower head) in millimeters.")
    head_diameter: Optional[Measurements] = Field(None, description="The measured diameters of the capitulum (flower head) in millimeters.")
    
    bur_length: Optional[Measurements] = Field(None, description="The measured length of the bur in millimeters.")
    tepal_length: Optional[Measurements] = Field(None, description="The measured length of the tepal in millimeters.")
    staminate_tepal_length: Optional[Measurements] = Field(None, description="The measured length of the staminate tepal in millimeters.")
    pistillate_tepal_length: Optional[Measurements] = Field(None, description="The measured length of the pistillate tepal in millimeters.")
      
    ray_length: Optional[Measurements] = Field(None, description="The measured length of the ray in millimeters.")
    ray_width: Optional[Measurements] = Field(None, description="The measured width of the ray in millimeters.")
    
    florets_length: Optional[Measurements] = Field(None, description="The measured length of the florets in millimeters.")
    
    involucre_length: Optional[Measurements] = Field(None, description="Involucre length in millimeters")
    involucre_width: Optional[Measurements] = Field(None, description="Involucre width in millimeters")
    staminate_involucre_length: Optional[Measurements] = Field(None, description="Staminate involucre length in millimeters")
    pistilate_involucre_length: Optional[Measurements] = Field(None, description="Pistillate involucre length in millimeters")   
    
    bract_length: Optional[Measurements] = Field(None, description="Bract length in millimeters")
    bract_width: Optional[Measurements] = Field(None, description="Bract width in millimeters")
    bract_lower_length: Optional[Measurements] = Field(None, description="Lower bract length in millimeters")
    bract_outer_length: Optional[Measurements] = Field(None, description="Outer bract length in millimeters")
    
    bracteoles_length: Optional[Measurements] = Field(None, description="Bracteoles length in millimeters")
    bracteoles_width: Optional[Measurements] = Field(None, description="Bracteole width in millimeters")
    
    pedicel_length: Optional[Measurements] = Field(None, description="Pedicel length in millimeters")
    pedicel_width: Optional[Measurements] = Field(None, description="Pedicel width in millimeters")
    staminate_pedicel_length: Optional[Measurements] = Field(None, description="Staminate pedicel length in millimeters")
    pistillate_pedicel_length: Optional[Measurements] = Field(None, description="Pistillate pedicel length in millimeters")
    staminate_pedicel_width: Optional[Measurements] = Field(None, description="Staminate pedicel width in millimeters")
    pistillate_pedicel_width: Optional[Measurements] = Field(None, description="Pistillate pedicel width in millimeters") 
    
    hypanthium_length: Optional[Measurements] = Field(None, description="Hypanthium length in millimeters")
    hypanthium_width: Optional[Measurements] = Field(None, description="Hypanthium width in millimeters")
    
    peduncle_length: Optional[Measurements] = Field(None, description="Peduncle length in millimeters")
    peduncle_width: Optional[Measurements] = Field(None, description="Peduncle width in millimeters")
    staminate_peduncle_length: Optional[Measurements] = Field(None, description="Staminate peduncle length in millimeters")
    staminate_peduncle_width: Optional[Measurements] = Field(None, description="Staminate peduncle width in millimeters")
    pistillate_peduncle_length: Optional[Measurements] = Field(None, description="Pistillate peduncle length in millimeters")
    pistillate_peduncle_width: Optional[Measurements] = Field(None, description="Pistillate peduncle width in millimeters")

    spathe_width: Optional[Measurements] = Field(None, description="Spathe width dimensions in millimeters")
    spathe_length: Optional[Measurements] = Field(None, description="Spathe length dimensions in millimeters")
    spadix_length: Optional[Measurements] = Field(None, description="Spadix length dimensions in millimeters")
    
    perianth_width: Optional[Measurements] = Field(None, description="Perianth width dimensions in millimeters")
    perianth_length: Optional[Measurements] = Field(None, description="Perianth length dimensions in millimeters")   
    perianth_outer_width: Optional[Measurements] = Field(None, description="Outer perianth width dimensions in millimeters")
    perianth_outer_length: Optional[Measurements] = Field(None, description="Outer perianth length dimensions in millimeters") 
    perianth_inner_width: Optional[Measurements] = Field(None, description="Inner perianth width dimensions in millimeters")
    perianth_inner_length: Optional[Measurements] = Field(None, description="Inner perianth length dimensions in millimeters") 
    
    perianth_lobes_width: Optional[Measurements] = Field(None, description="Perianth lobes width dimensions in millimeters")
    perianth_lobes_length: Optional[Measurements] = Field(None, description="Perianth lobes length dimensions in millimeters") 
    perianth_tube_length: Optional[Measurements] = Field(None, description="Perianth tube length dimensions in millimeters") 
    pistillate_perianth_tube_length: Optional[Measurements] = Field(None, description="Pistillate perianth tube length dimensions in millimeters") 
    staminate_perianth_tube_length: Optional[Measurements] = Field(None, description="Staminate perianth tube length dimensions in millimeters") 
    
    pappus_length: Optional[Measurements] = Field(None, description="Pappus length in millimeters")
    umbellet_length: Optional[Measurements] = Field(None, description="Umbellet length in millimeters")
    labellum_width: Optional[Measurements] = Field(None, description="Labellum width dimensions in millimeters")
    labellum_length: Optional[Measurements] = Field(None, description="Labellum length dimensions in millimeters")
    
    calyx_length: Optional[Measurements] = Field(None, description="Calyx length in millimeters")
    calyx_width: Optional[Measurements] = Field(None, description="Calyx width in millimeters")
    calyx_teeth_length: Optional[Measurements] = Field(None, description="Calyx teeth length in millimeters")
    calyx_teeth_width: Optional[Measurements] = Field(None, description="Calyx teeth width in millimeters")
    calyx_lobes_length: Optional[Measurements] = Field(None, description="Calyx lobe length in millimeters")
    calyx_lobes_width: Optional[Measurements] = Field(None, description="Calyx lobe width in millimeters")
    
    upper_calyx_length: Optional[Measurements] = Field(None, description="Upper calyx length in millimeters")
    lower_calyx_length: Optional[Measurements] = Field(None, description="Lower calyx length in millimeters")
    
    inner_calyx_lobes_length: Optional[Measurements] = Field(None, description="Inner calyx lobes length in millimeters")
    inner_calyx_lobes_width: Optional[Measurements] = Field(None, description="Inner calyx lobes width in millimeters")
    outer_calyx_lobes_length: Optional[Measurements] = Field(None, description="Outer calyx lobes length in millimeters")
    outer_calyx_lobes_width: Optional[Measurements] = Field(None, description="Outer calyx lobes width in millimeters")
    
    calyx_tube_length: Optional[Measurements] = Field(None, description="Calyx tube length in millimeters")
    calyx_tube_width: Optional[Measurements] = Field(None, description="Calyx tube width in millimeters")
    
    male_calyx_length: Optional[Measurements] = Field(None, description="Male calyx length in millimeters")
    male_calyx_width: Optional[Measurements] = Field(None, description="Male calyx width in millimeters")
    
    male_calyx_lobes_length: Optional[Measurements] = Field(None, description="Male calyx length in millimeters")
    male_calyx_lobes_width: Optional[Measurements] = Field(None, description="Male calyx width in millimeters")
    
    female_calyx_length: Optional[Measurements] = Field(None, description="female calyx length in millimeters")
    female_calyx_width: Optional[Measurements] = Field(None, description="female calyx width in millimeters")
    
    female_calyx_lobes_length: Optional[Measurements] = Field(None, description="Female calyx length in millimeters")
    female_calyx_lobes_width: Optional[Measurements] = Field(None, description="Female calyx width in millimeters")
    
    male_calyx_lobes_length_inner: Optional[Measurements] = Field(None, description="Male calyx inner lobe length in millimeters")
    male_calyx_lobes_length_outer: Optional[Measurements] = Field(None, description="Male calyx outer length in millimeters")
    female_calyx_lobes_length_inner: Optional[Measurements] = Field(None, description="Female calyx inner lobe length in millimeters")
    female_calyx_lobes_length_outer: Optional[Measurements] = Field(None, description="Female calyx outer length in millimeters")
    
    male_calyx_lobes_width_outer: Optional[Measurements] = Field(None, description="Male calyx outer lobe width in millimeters")
    male_calyx_tube_length: Optional[Measurements] = Field(None, description="Male calyx tube length in millimeters")

    female_calyx_lobes_width_inner: Optional[Measurements] = Field(None, description="Female calyx inner lobe width in millimeters")
    female_calyx_lobes_width_outer: Optional[Measurements] = Field(None, description="Female calyx outer lobe width in millimeters")
    female_calyx_tube_length: Optional[Measurements] = Field(None, description="Female calyx tube length in millimeters")
    
    inner_calyx_length: Optional[Measurements] = Field(None, description="Inner calyx length in millimeters")
    outer_calyx_length: Optional[Measurements] = Field(None, description="Outer calyx length in millimeters")

    corolla_length: Optional[Measurements] = Field(None, description="Corolla length in millimeters")
    corolla_width: Optional[Measurements] = Field(None, description="Corolla width in millimeters")
    
    corolla_length: Optional[Measurements] = Field(None, description="Corolla length in millimeters")
    corolla_width: Optional[Measurements] = Field(None, description="Corolla width in millimeters")
    corolla_tube_length: Optional[Measurements] = Field(None, description="Corolla tube length in millimeters")
    corolla_tube_width: Optional[Measurements] = Field(None, description="Corolla tube width in millimeters")
    corolla_lobes_length: Optional[Measurements] = Field(None, description="Corolla lobes length in millimeters")
    corolla_lobes_width: Optional[Measurements] = Field(None, description="Corolla lobes width in millimeters")
    
    
    upper_corolla: Optional[Measurements] = Field(None, description="Upper corolla length in millimeters")
    lower_corolla: Optional[Measurements] = Field(None, description="Lower corolla length in millimeters")
    upper_corolla_lobes_length: Optional[Measurements] = Field(None, description="Upper corolla lobes length in millimeters")
    lower_corolla_lobes_length: Optional[Measurements] = Field(None, description="Lower corolla lobes length in millimeters")
    
    corolla_lip: Optional[Measurements] = Field(None, description="Corolla lip length in millimeters")
    
    staminate_corolla_length: Optional[Measurements] = Field(None, description="Staminate corolla length in millimeters")
    pistillate_corolla_length: Optional[Measurements] = Field(None, description="Pistillate corolla length in millimeters")
    
    staminate_corolla_tube_length: Optional[Measurements] = Field(None, description="Staminate corolla tube length in millimeters")
    pistillate_corolla_tube_length: Optional[Measurements] = Field(None, description="Pistillate corolla tube length in millimeters")

    staminate_corolla_tube_width: Optional[Measurements] = Field(None, description="Staminate corolla tube length in millimeters")
    pistillate_corolla_tube_width: Optional[Measurements] = Field(None, description="Pistillate corolla tube length in millimeters")    
    
    female_corolla_lobes_length: Optional[Measurements] = Field(None, description="Female corolla lobes length in millimeters") 
    female_corolla_lobes_width:  Optional[Measurements] = Field(None, description="Female corolla lobes width in millimeters") 
    male_corrola_lobes_length: Optional[Measurements] = Field(None, description="Male corolla lobes length in millimeters") 
    male_corrola_lobes_width: Optional[Measurements] = Field(None, description="Male corolla lobes width in millimeters") 
    
    fruit_length: Optional[Measurements] = Field(None, description="Fruit length in centimeters") 
    fruit_width: Optional[Measurements] = Field(None, description="Fruit width in centimeters") 
    fruit_diameter: Optional[Measurements] = Field(None, description="Fruit diameter in centimeters") 
    
    
    seeds_perfruit: Optional[Measurements] = Field(None, description="Number of seeds per fruit") 
    
    seed_length: Optional[Measurements] = Field(None, description="Seed length in centimeters") 
    seed_width: Optional[Measurements] = Field(None, description="Seed width in centimeters") 
    seed_diameter: Optional[Measurements] = Field(None, description="Seed diameter in centimeters") 


In [3]:
## Input to base64 format to be intepreted by extraction agents (chatbots)
with open(f"{photo_dir}/Acanthaceae_Dicliptera_chinensis.jpeg", "rb") as image_file:
            base64_image = base64.b64encode(image_file.read()).decode("utf-8")

## Setting the overall framework for how the extraction agent operates across all interactions
system_prompt  = """
You are an expert plant taxonomist. Please analyze this image and return the details according to the schema provided.
You are exhaustive; you include ALL the details mentioned. Do not make any assumptions about the data and do not try to
interpret what is not obvious from the text. When extracting information, ensure that you return a structured response in JSON format matching the `HawaiianPlant` schema. """

## Prompt to set specific focus for each extraction agent interaction (can be dynamic as need be)
user_prompt= "Transcribe the plant information you see in this image"


## DeepSeek Model Extraction (R1 distill llama 70B) from text to Hawaiian Plant Schema

In [4]:
# ## Setting deepseek LLM as extraction agent
# ## Data to be extracted in HawaiianPlant schema
# groq_extraction_agent = Agent(
#     model="groq:deepseek-r1-distill-llama-70b",
#     retries=3,
#     result_type=HawaiianPlant,
#     system_prompt = system_prompt,
#     # Had to raise temperature to 0.5
#     model_settings = {'temperature': 0.5}
# )

# ## Setting variables and extracting data
# r = await groq_extraction_agent.run(text)
# deepseek_llama = r.data


## Data Extraction for Multiple Models

In [11]:
family = 'Araliaceae'
genus = 'Cheirodendron'
species = 'forbesii'

## Input to base64 format to be intepreted by extraction agents (chatbots)
with open(f"{photo_dir}/{family}_{genus}_{species}.jpeg", "rb") as image_file:
            base64_image = base64.b64encode(image_file.read()).decode("utf-8")

## Setting the overall framework for how the extraction agent operates across all interactions
system_prompt  = """
You are an expert plant taxonomist. Please analyze this image and return the details according to the schema provided.
You are exhaustive; you include ALL the details mentioned. Do not make any assumptions about the data and do not try to
interpret what is not obvious from the text. When extracting information, ensure that you return a structured response in JSON format matching the `HawaiianPlant` schema. """

## Prompt to set specific focus for each extraction agent interaction (can be dynamic as need be)
user_prompt= "Transcribe the plant information you see in this image"

## Setting extraction agent to be gpt-4o
## System prompt is being set to specifically transcribe information
openai_img_to_text_transcription_agent = Agent(
    model="openai:gpt-4o",
    result_type=str,
    system_prompt = "You are a vision model capable of accurately performing OCR on an image",
)

## Data to be extracted from input image
image_urls = [
    f"data:image/png;base64,{base64_image}",
]

## Setting chat parameters: low detail (for efficiency) from input image
image_params = [
    ChatCompletionContentPartImageParam(
        type='image_url', 
        image_url=ImageURL(url=url, detail='low')
    ) for url in image_urls
]

## Setting chat prompt to 'user_prompt'
msg_open_ai = [
            ChatCompletionContentPartTextParam(text="Convert this to text. Don't miss any text.", type='text'),
            *image_params
]

## Running data extraction
r = await openai_img_to_text_transcription_agent.run(msg_open_ai)

## Output to 'text'

if len(r.data.split("---")) == 1:
    text = r.data.split("---")[0]
else:
    text = r.data.split("---")[1]
print(text)

## Setting extraction agent to be gpt-4o
## Extracted data will be formatted according to HawaiianPlant schema
openai_extraction_agent = Agent(
    model="openai:gpt-4o",
    result_type=HawaiianPlant,
    system_prompt = system_prompt,
)

## Data to be extracted from input image
image_urls = [
    f"data:image/png;base64,{base64_image}",
]

## Setting chat parameters: low detail (for efficiency) from input image
image_params = [
    ChatCompletionContentPartImageParam(
        type='image_url', 
        image_url=ImageURL(url=url, detail='low')
    ) for url in image_urls
]

## Setting chat prompt to 'user_prompt'
msg_open_ai = [
            ChatCompletionContentPartTextParam(text=user_prompt, type='text'),
            *image_params
]

## Running data extraction
r = await openai_extraction_agent.run(msg_open_ai)

## Setting results to 'gpt_4o_output'
gpt_4o_output = r.data

print('Done: gpt_4o_output')

## Setting extraction agent to be Claude Sonnet model
## Extracted data will be formatted according to HawaiianPlant schema
sonnet_extraction_agent = Agent(
    model="anthropic:claude-3-5-sonnet-latest",
    result_type=HawaiianPlant,
    system_prompt = system_prompt,
    model_settings = {'temperature': 0.2}

)

## Setting context
msg_claude = [
    ChatCompletionContentPartTextParam(text=user_prompt, type='text'),
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": f"{base64_image}",
                    }
                },
    
]

## Extracting data from image
r = await sonnet_extraction_agent.run(msg_claude)
sonnet_output = r.data

print('Done: sonnet_output')


## Setting system prompt
gptmini_extraction_agent = Agent(
    model="openai:o3-mini",
    result_type=HawaiianPlant,
    system_prompt = system_prompt,
)

## Extracting output from extracted text
r = await gptmini_extraction_agent.run(text)

## Setting output
gpt_mini_output = r.data

print('Done: miniGPT ')


3. Cheirodendron forbesii (Sherff) Lowry
Jc. Kaduaena Kraj. var. Forbesil Sherff
(enden)

Trees 3—4.5 m tall. Leaves I (occasionally 3 on juvenile leaves), subcoriaceous, elliptic to ovate, 5—12 cm long and less than
9.5 cm wide, turning yellow in upper
surface in dried material (vs. green on juve-
nile leaves), margins entire, apex rounded-
acute to acuminate, base narrowly cuneate
to attenuate, petioles 3—7(—10) cm long;
juvenile leaves, petioles 0.5—1.5 cm long,
suberect. Inflorescences up to 9 cm long,
pedicels 3—6 mm long; ovary (4)5-carpel-
late; styles (4)5, 1—1.5 mm long, connate (vs.
Y). The length into a concaniformed struc-
lopodium, the distinct arms divergent to-
spreading. Fruit globose, weakly (4)5-lobed,
(5-lobed) ribbed when dry, 4—5 mm long
(in stylés). Occurring in wet forest, 600—
900 m, Mount Ka’ala, north fork of Ka-ma-
koan Stream, Kalalau, Makaha, and
Polihale. These Hawaiian Mountains (en-
dimie).

Cheirodendron kauai Krajewski, nov.
Cheirodendron macrocarpom v

## Reformat Manually Extracted Data

In [12]:
df= pd.read_csv("/Users/williamharrigan/Desktop/Github/ai_wagner_trait_data_extraction/files/man_extract.csv")
df.columns = df.columns.str.lower()

codes = pd.read_csv('/Users/williamharrigan/Desktop/Github/ai_wagner_trait_data_extraction/files/wagner_codes.csv')
codes.columns = codes.columns.str.lower()

type_cols = [col for col in df.columns if '_type_' in col]
col_groups = {}
for col in type_cols:
    prefix = '_'.join(col.split('_')[:-1])
    # print(prefix)
    if prefix not in col_groups:
        col_groups[prefix] = []
    col_groups[prefix].append(col)
    # print(col)

# # Create new collapsed columns
for prefix, cols in col_groups.items():
    df[prefix] = df.apply(lambda row: [col.split('_')[-1].upper() for col in cols if row[col] == 1], axis=1)
    
# Drop the original type columns
df.drop(columns=type_cols, inplace=True)

df['hawaiian_name'] = df[['hawaiian_name_1', 'hawaiian_name_2', 'hawaiian_name_3', 'hawaiian_name_4']].apply(
    lambda row: {x for x in row if pd.notna(x)}, axis=1
)

# Drop original columns if needed
df.drop(columns=['hawaiian_name_1', 'hawaiian_name_2', 'hawaiian_name_3', 'hawaiian_name_4'], inplace=True)

def collapse_measurements(df):
    # Group columns by their base name (everything before _min, _max, etc.)
    base_columns = {}
    
    for col in df.columns:
        parts = col.split('_')
        if len(parts) > 2:
            suffix = parts[-1]
            if suffix in ['exmin', 'min', 'max', 'exmax']:
                base_name = '_'.join(parts[:-1])
                if base_name not in base_columns:
                    base_columns[base_name] = []
                base_columns[base_name].append(col)
    
    # Columns to copy directly (non-measurement columns)
    non_measurement_cols = [col for col in df.columns 
                           if not any(col in cols for cols in base_columns.values())]
    
    # Start with non-measurement columns
    result_df = df[non_measurement_cols].copy()
    
    # Process each group of measurement columns
    collapsed_data = {}
    
    for base_name, cols in base_columns.items():
        # Extract just the name without the unit
        name_parts = base_name.split('_')
        unit = name_parts[-1]
        col_name = '_'.join(name_parts[:-1])
        
        # Create dictionaries for each row
        collapsed_data[col_name] = []
        
        for _, row in df.iterrows():
            measurement_dict = {
                'exmin': row[f"{base_name}_exmin"] if f"{base_name}_exmin" in cols else None,
                'min': row[f"{base_name}_min"] if f"{base_name}_min" in cols else None,
                'max': row[f"{base_name}_max"] if f"{base_name}_max" in cols else None,
                'exmax': row[f"{base_name}_exmax"] if f"{base_name}_exmax" in cols else None,
                'unit': unit
            }
            collapsed_data[col_name].append(measurement_dict)
    
    # Convert to DataFrame and concatenate with result_df
    collapsed_df = pd.DataFrame(collapsed_data)
    result_df = pd.concat([result_df, collapsed_df], axis=1)
    
    return result_df

# Example usage
# df is your original dataframe with columns like stem_height_m_exmin, etc.
new_df = collapse_measurements(df)

# Function to create a mapping dictionary from the codes dataframe
def create_mapping_dict(df, column_name):
    mapping = {}
    for entry in df[column_name].dropna():
        if isinstance(entry, str) and '=' in entry:  # Ensure valid key-value pairs
            key, value = entry.split('=', 1)
            mapping[key.strip()] = value.strip().lower()
    return mapping

# Generate mapping dictionaries for each relevant column
mapping_dicts = {col: create_mapping_dict(codes, col) for col in codes.columns if col in new_df.columns}

# Function to apply the mapping to new_df
def map_values(value, mapping):
    if pd.isna(value):
        return value  # Keep NaNs unchanged
    if isinstance(value, str):
        split_values = value.split(' to ')  # Handle cases like "G to Hi"
        mapped_values = [mapping.get(v.strip(), v.strip()) for v in split_values]
        return ' to '.join(mapped_values)
    return value  # Return unchanged if it's not a string

# Apply mappings to new_df
for col in mapping_dicts:
    new_df[col] = new_df[col].astype(str).apply(lambda x: map_values(x, mapping_dicts[col]))


In [13]:
new_df.head()

,family,genus,species,common_name,wagner_pg_number,description,infraspecific_epithet,stem_hair_type,phyllotaxy_type,leaf_hair_description,...,female_corolla_lobes_width,male_corrola_lobes_length,male_corrola_lobes_width,fruit_length,fruit_width,fruit_diameter,seeds_perfruit,seed_length,seed_width,seed_diameter
0,Asteraceae,Ambrosia,artemisiifolia,common ragweed,pg 256-257,Dicots,NaN,hirsute,O below and A above,NaN,...,"{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': None, 'min': nan, 'max': nan, 'exmax...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': None, 'min': nan, 'max': nan, 'exmax...","{'exmin': None, 'min': nan, 'max': nan, 'exmax..."
1,Asteraceae,Dubautia,laxa,na`ena`e pua melemele,"pg 292-295,301",Dicots,pseudoplantaginea,glabrous (smooth) to hispid,whorled,glabrous,...,"{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': 2.0, 'max': 3.0, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': None, 'min': nan, 'max': nan, 'exmax...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': None, 'min': nan, 'max': nan, 'exmax...","{'exmin': None, 'min': nan, 'max': nan, 'exmax..."
2,Asteraceae,Tetramolopium,filiforme,no common name,"pg 361-362, 365, 366",Dicots,polyphyllum,glabrous (smooth),alternate,NaN,...,"{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': 2.0, 'max': 2.7, 'exmax'...","{'exmin': nan, 'min': 0.6, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': None, 'min': nan, 'max': nan, 'exmax...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': None, 'min': nan, 'max': nan, 'exmax...","{'exmin': None, 'min': nan, 'max': nan, 'exmax..."
3,Asteraceae,Encelia,farinosa,brittle bush,pg 312-313,Dicots,NaN,puberulent,alternate,densely white tomentose,...,"{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': 4.0, 'max': 4.5, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': None, 'min': nan, 'max': nan, 'exmax...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': None, 'min': nan, 'max': nan, 'exmax...","{'exmin': None, 'min': nan, 'max': nan, 'exmax..."
4,Aristolochiaceae,Aristolochia,littoralis,calico flower,"pg 237-238,239",Dicots,NaN,nan,alternate,NaN,...,"{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': 45.0, 'exmax...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': nan, 'min': nan, 'max': nan, 'exmax'...","{'exmin': None, 'min': nan, 'max': nan, 'exmax...","{'exmin': nan, 'min': nan, 'max': 0.6, 'exmax'...","{'exmin': None, 'min': nan, 'max': nan, 'exmax...","{'exmin': None, 'min': nan, 'max': nan, 'exmax..."


## Compare Manual vs Automated Extractions

In [14]:
class AreAnntoationsEqual(BaseModel):
    are_equal: bool = Field(..., description="Are the two annotations equal")
    justification: str = Field(..., description="Justification of the propose value for are_equal. Justifications should be as concise as possible.")
    

## Setting the prompt and model for validation agent
validation_agent = Agent(
    model="groq:llama-3.3-70b-specdec",
    result_type=AreAnntoationsEqual,
    system_prompt = """You are an expert taxonomist. You are comparing the outcome of a manually extracted result versus an automatically extracted result. You need to compare the automatic results and determine whether the result is synonymous or equal the manual one; taking into consideration
    linguisitc and formatting nuances. Your answer is whether the two results are similar True/False and a justificaiton for your answer""",
)

## Models to compare
# models = ["gpt_4o_output", "gpt_mini_output", "sonnet_output", "deepseek_llama"]
models = ["gpt_4o_output", "gpt_mini_output", "sonnet_output"]

In [15]:
ground_truth = new_df[(new_df['family'] == family) & 
                      (new_df['genus'] == genus) & 
                      (new_df['species'] == species)].to_dict(orient='records')[0]

model_outputs = {}
model_outputs['gpt_4o_output'] = dict(gpt_4o_output)
model_outputs['gpt_mini_output'] = dict(gpt_mini_output)
model_outputs['sonnet_output'] = dict(sonnet_output)

In [16]:
ground_truth.items()

dict_items([('family', 'Araliaceae'), ('genus', 'Cheirodendron'), ('species', 'forbesii'), ('common_name', 'olapa, lapalapa'), ('wagner_pg_number', 'pg 225,227'), ('description', 'Dicots'), ('infraspecific_epithet', nan), ('stem_hair_type', 'glabrous (smooth)'), ('phyllotaxy_type', 'opposite'), ('leaf_hair_description', 'glabrous'), ('leaf_hair_upper_description', nan), ('leaf_hair_lower_description', nan), ('breeding_type', 'andromonoecious'), ('inflorescence_type', 'umbel'), ('ray_color', nan), ('floret_color', nan), ('spathe_color', nan), ('perianth_outer_color', nan), ('perianth_inner_color', nan), ('perianth_color', nan), ('labellum_color', nan), ('corolla_type', 'triangular to ovate to oblong'), ('staminate_corolla_type', nan), ('pistillate_corolla_type', nan), ('corolla_color', 'Purple externally, greenish within'), ('fruit_type', 'drupe'), ('ploidy', nan), ('chromosome_number', nan), ('average_chromosome_number', nan), ('origin', 'endemic'), ('fed_status', 'NS'), ('status', 'Se

In [18]:

for gt_key, gt_value in ground_truth.items():
    print(f"Groundtruth {gt_key}: {gt_value}")
    
    for model in model_outputs.keys():
        auto_value = model_outputs[model].get(gt_key, "N/A")  # Handle missing keys safely

        user_prompt = f"""manually annotated {gt_key}: {gt_value}
automatically annotated {gt_key}: {auto_value}"""
        if gt_value == 'nan' and auto_value == 'None':
            print(f"Model {model}: {auto_value} \t Accurate output: True (Both NaN)")
        else:
            try:
                result = await validation_agent.run(user_prompt)  # Ensure `await` is used correctly
                print(f"Model {model}: {auto_value} \t Accurate output: {result.data.are_equal}")
                print(f"Justification: {result.data.justification}")  # Print the justification
            except Exception as e:
                print(f"Error processing model {model} for key {gt_key}: {e}")

    print()


Groundtruth family: Araliaceae
